In [ ]:
# pip install mediapipe

In [ ]:
import numpy as np
import pandas as pd
import mediapipe
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import  RobustScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# from google.colab import drive
# drive.mount('/content/AFLW2000')

Mounted at /content/AFLW2000


In [ ]:
df = pd.read_csv("/content/AFLW2000/MyDrive/Data_mediapipe_Last.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,930,931,932,933,934,935,936,937,938,939
0,0.938597,1.007386,1.098770,0.182021,0.763815,0.510631,0.609138,-0.391791,1.076836,-0.031944,...,0.308671,-0.800266,0.490267,-1.006096,0.482867,-1.089393,-0.974573,0.510970,-0.369454,image03196
1,0.620653,1.375399,1.325659,0.525301,0.589380,0.879985,0.836262,-0.283631,1.391070,0.246671,...,0.390860,-0.863887,0.437160,-1.010319,0.427355,-1.148040,-1.228429,-0.368714,0.177511,image03161
2,-0.200145,0.945523,-0.232793,0.611262,-0.213700,0.632172,-0.386228,-0.038489,-0.240714,0.466111,...,0.098146,-0.705439,1.153548,-1.127718,1.229906,-1.216391,-0.040907,-0.474087,0.019801,image03238
3,0.245179,0.390212,0.669888,-0.102398,0.258672,0.086786,0.374627,-0.560250,0.706987,-0.251751,...,0.174222,-0.944195,0.335404,-1.086590,0.352655,-1.176436,-0.985426,-0.379067,0.086955,image03249
4,-0.231091,0.608912,-0.168474,0.137574,-0.230430,0.238288,-0.368546,-0.476936,-0.161499,-0.028239,...,-0.037472,-0.993423,0.832600,-1.310196,0.895753,-1.368230,0.039903,-0.192543,-0.027199,image03217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,0.096465,1.103705,-0.078571,0.729682,0.013125,0.793496,-0.274523,0.219890,-0.121227,0.594823,...,0.095882,-0.301489,0.951424,-0.702999,1.009162,-0.780140,0.206935,-0.219317,-0.202272,image00856
1819,-0.026975,0.472753,-0.188752,0.196186,-0.057657,0.292424,-0.182060,-0.095267,-0.198417,0.100473,...,0.176497,-0.290683,0.756945,-0.397687,0.822751,-0.433359,0.580349,-0.046365,-0.023616,image00847
1820,2.354778,0.784182,2.408616,0.335732,2.304485,0.496784,2.187844,-0.035539,2.404536,0.208834,...,2.261684,-0.292790,2.736815,-0.448154,2.779947,-0.496736,-0.617419,0.137163,-0.091421,image00837
1821,0.542091,0.594646,0.214526,0.240460,0.489729,0.345595,0.221053,-0.097081,0.169589,0.126626,...,0.661764,-0.457418,1.296870,-0.812498,1.355925,-0.912849,0.983607,0.236384,-0.383734,image00861


In [ ]:
features = df.iloc[:,:936]
labels = df.iloc[:,936:939]

In [ ]:
features

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,0.938597,1.007386,1.098770,0.182021,0.763815,0.510631,0.609138,-0.391791,1.076836,-0.031944,...,0.225525,-0.883184,0.238485,-0.828632,0.308671,-0.800266,0.490267,-1.006096,0.482867,-1.089393
1,0.620653,1.375399,1.325659,0.525301,0.589380,0.879985,0.836262,-0.283631,1.391070,0.246671,...,0.268128,-1.014951,0.284161,-0.929932,0.390860,-0.863887,0.437160,-1.010319,0.427355,-1.148040
2,-0.200145,0.945523,-0.232793,0.611262,-0.213700,0.632172,-0.386228,-0.038489,-0.240714,0.466111,...,0.253086,-0.885040,0.153967,-0.795218,0.098146,-0.705439,1.153548,-1.127718,1.229906,-1.216391
3,0.245179,0.390212,0.669888,-0.102398,0.258672,0.086786,0.374627,-0.560250,0.706987,-0.251751,...,0.115910,-1.037306,0.116482,-0.986301,0.174222,-0.944195,0.335404,-1.086590,0.352655,-1.176436
4,-0.231091,0.608912,-0.168474,0.137574,-0.230430,0.238288,-0.368546,-0.476936,-0.161499,-0.028239,...,0.064317,-1.117025,-0.009008,-1.049609,-0.037472,-0.993423,0.832600,-1.310196,0.895753,-1.368230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,0.096465,1.103705,-0.078571,0.729682,0.013125,0.793496,-0.274523,0.219890,-0.121227,0.594823,...,0.230956,-0.435830,0.146992,-0.363535,0.095882,-0.301489,0.951424,-0.702999,1.009162,-0.780140
1819,-0.026975,0.472753,-0.188752,0.196186,-0.057657,0.292424,-0.182060,-0.095267,-0.198417,0.100473,...,0.293496,-0.327652,0.230963,-0.305764,0.176497,-0.290683,0.756945,-0.397687,0.822751,-0.433359
1820,2.354778,0.784182,2.408616,0.335732,2.304485,0.496784,2.187844,-0.035539,2.404536,0.208834,...,2.288594,-0.339409,2.257112,-0.308872,2.261684,-0.292790,2.736815,-0.448154,2.779947,-0.496736
1821,0.542091,0.594646,0.214526,0.240460,0.489729,0.345595,0.221053,-0.097081,0.169589,0.126626,...,0.826545,-0.556170,0.744546,-0.497788,0.661764,-0.457418,1.296870,-0.812498,1.355925,-0.912849


In [ ]:
labels

,936,937,938
0,-0.974573,0.510970,-0.369454
1,-1.228429,-0.368714,0.177511
2,-0.040907,-0.474087,0.019801
3,-0.985426,-0.379067,0.086955
4,0.039903,-0.192543,-0.027199
...,...,...,...
1818,0.206935,-0.219317,-0.202272
1819,0.580349,-0.046365,-0.023616
1820,-0.617419,0.137163,-0.091421
1821,0.983607,0.236384,-0.383734


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, shuffle=True, test_size=0.2,random_state=0)

In [ ]:
X_tst, X_val, y_tst, y_val = train_test_split(X_test, y_test, shuffle=True, test_size=0.5,random_state=0)

# **RandomForestRegressor** (Roll Saved Model)


In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
rf_clf = make_pipeline(StandardScaler(),
                       PCA(n_components=0.99),
                        RandomForestRegressor(n_estimators=300 , random_state=42))

In [ ]:
rf_clf.fit(X_train, y_train.iloc[:, -1])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.99)),
                ('randomforestregressor',
                 RandomForestRegressor(n_estimators=300, random_state=42))])

In [ ]:
print("Score \n")
print("Train==>  " ,rf_clf.score(X_train, y_train.iloc[:, -1]))
print("Val==>  " ,rf_clf.score(X_val, y_val.iloc[:, -1]))
print("test==>  " ,rf_clf.score(X_tst, y_tst.iloc[:, -1]))

Score 

Train==>   0.9193510130404324
Val==>   0.8173554661949588
test==>   0.7755632389163607


In [ ]:
predictions_train = rf_clf.predict(X_train)
predictions_val = rf_clf.predict(X_val)
predictions_test = rf_clf.predict(X_tst)

In [ ]:
print("\n R2_Score \n")
r2_train = r2_score(y_train.iloc[:, -1], predictions_train)
r2_val = r2_score(y_val.iloc[:, -1], predictions_val)
r2_tst = r2_score(y_tst.iloc[:, -1], predictions_test)
print("Train ==>  " , r2_train)
print("Val ==>  " , r2_val)
print("test ==>  " , r2_tst)


 R2_Score 

Train ==>   0.9193510130404324
Val ==>   0.8173554661949588
test ==>   0.7755632389163607


In [ ]:
import pickle 

with open('roll_model.pkl', 'wb') as f:
    pickle.dump(rf_clf, f)

---



---


# ***Trials***


---



---



# SVR

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
rf_clf_svr = make_pipeline(MinMaxScaler(),
                       PCA(n_components=0.99),
                        SVR(kernel='rbf', C=9.0, epsilon=0.0000089))

In [ ]:
rf_clf_svr.fit(X_train, y_train.iloc[:, -1])

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pca', PCA(n_components=0.99)),
                ('svr', SVR(C=9.0, epsilon=8.9e-06))])

In [ ]:
print("Score \n")
print("Train==>  " ,rf_clf_svr.score(X_train, y_train.iloc[:, -1]))
print("Val==>  " ,rf_clf_svr.score(X_val, y_val.iloc[:, -1]))
print("test==>  " ,rf_clf_svr.score(X_tst, y_tst.iloc[:, -1]))

Score 

Train==>   0.6067030365147663
Val==>   0.7183034221723155
test==>   0.7505488424689527


In [ ]:
predictions_train = rf_clf_svr.predict(X_train)
predictions_val = rf_clf_svr.predict(X_val)
predictions_test = rf_clf_svr.predict(X_tst)

In [ ]:
print("\n R2_Score \n")
r2_train = r2_score(y_train.iloc[:, -1], predictions_train)
r2_val = r2_score(y_val.iloc[:, -1], predictions_val)
r2_tst = r2_score(y_tst.iloc[:, -1], predictions_test)
print("Train ==>  " , r2_train)
print("Val ==>  " , r2_val)
print("test ==>  " , r2_tst)


 R2_Score 

Train ==>   0.6067030365147663
Val ==>   0.7183034221723155
test ==>   0.7505488424689527


# DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
rf_clf_DTR = make_pipeline(RobustScaler(),
                       PCA(n_components=0.99),
                      DecisionTreeRegressor(max_depth=10,random_state=42))

In [ ]:
rf_clf_DTR.fit(X_train, y_train.iloc[:, -2])

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('pca', PCA(n_components=0.99)),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=10, random_state=42))])

In [ ]:
print("Score \n")
print("Train==>  " ,rf_clf_DTR.score(X_train, y_train.iloc[:, -2]))
print("Val==>  " ,rf_clf_DTR.score(X_val, y_val.iloc[:, -2]))
print("test==>  " ,rf_clf_DTR.score(X_tst, y_tst.iloc[:, -2]))

Score 

Train==>   0.9742349451667841
Val==>   0.49131312699746976
test==>   0.2632733066090267


In [ ]:
predictions_train = rf_clf_DTR.predict(X_train)
predictions_val = rf_clf_DTR.predict(X_val)
predictions_test = rf_clf_DTR.predict(X_tst)

In [ ]:
print("\n R2_Score \n")
r2_train = r2_score(y_train.iloc[:, -2], predictions_train)
r2_val = r2_score(y_val.iloc[:, -2], predictions_val)
r2_tst = r2_score(y_tst.iloc[:, -2], predictions_test)
print("Train ==>  " , r2_train)
print("Val ==>  " , r2_val)
print("test ==>  " , r2_tst)


 R2_Score 

Train ==>   0.9742349451667841
Val ==>   0.49131312699746976
test ==>   0.2632733066090267


# AdaBoostRegressor 

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
ad_clf = make_pipeline(StandardScaler(),
                       PCA(n_components=0.99),
                      AdaBoostRegressor(random_state=0, n_estimators=250, learning_rate=0.0000058))

In [ ]:
ad_clf.fit(X_train, y_train.iloc[:, -2])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.99)),
                ('adaboostregressor',
                 AdaBoostRegressor(learning_rate=5.8e-06, n_estimators=250,
                                   random_state=0))])

In [ ]:
print("Score \n")
print("Train==>  " ,ad_clf.score(X_train, y_train.iloc[:, -2]))
print("Val==>  " ,ad_clf.score(X_val, y_val.iloc[:, -2]))
print("test==>  " ,ad_clf.score(X_tst, y_tst.iloc[:, -2]))

Score 

Train==>   0.8636301766428937
Val==>   0.5886556792835276
test==>   0.5019150037816791


In [ ]:
predictions_train = ad_clf.predict(X_train)
predictions_val = ad_clf.predict(X_val)
predictions_test = ad_clf.predict(X_tst)

In [ ]:
print("\n R2_Score \n")
r2_train = r2_score(y_train.iloc[:, -2], predictions_train)
r2_val = r2_score(y_val.iloc[:, -2], predictions_val)
r2_tst = r2_score(y_tst.iloc[:, -2], predictions_test)
print("Train ==>  " , r2_train)
print("Val ==>  " , r2_val)
print("test ==>  " , r2_tst)


 R2_Score 

Train ==>   0.8636301766428937
Val ==>   0.5886556792835276
test ==>   0.5019150037816791
